In [11]:
from torch import nn, load as torch_load
from efficientnet_pytorch import model as efficientnet_model
import torch
from torchsummary import summary

In [6]:
output_dimensions = 5
data_dir = 'dataset'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
class EfficientNet(nn.Module):
    """
        Classe que implementa a arquitetura EfficientNet

        Parâmetros:
            backbone: str
                Nome do modelo de EfficientNet a ser utilizado
            output_dimensions: int
                Número de neuronios na camada de saída
    """
    def __init__(self, output_dimensions):
        super().__init__()
        self.efficient_net = efficientnet_model.EfficientNet.from_pretrained("efficientnet-b0")        
        self.fully_connected = nn.Linear(self.efficient_net._fc.in_features, output_dimensions)
        self.efficient_net._fc = nn.Identity()

    def extract(self, inputs):
        return self.efficient_net(inputs)

    def forward(self, inputs):
        x = self.extract(inputs)
        x = self.fully_connected(x)

        return x

In [9]:
model = EfficientNet(output_dimensions)
model.load_state_dict(
    torch_load(
        "../pre-trained-models/efficientnet-b0_segmented_fold_2_without_pen.pth", 
        weights_only=True,
        map_location=device
    )
)

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [ ]:
summary(model, (3, 1536, 1536))